In [18]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize          
from sklearn.feature_extraction import stop_words


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rosedennis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('../Data/final.csv')

In [4]:
df['subreddit'] = df['subreddit'].map({'Showerthoughts': 0, 'StonerPhilosophy': 1})
y = df['subreddit']

In [5]:
X = df['title']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)

In [19]:
class LemmaTokenizer(object): #https://stackoverflow.com/questions/47423854/sklearn-adding-lemmatizer-to-countvectorizer
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


In [20]:
cvec = CountVectorizer(tokenizer=LemmaTokenizer(),
                      stop_words = 'english',
                      lowercase = False)
cvec.fit(X_train)
X_train = cvec.transform(X_train)

/Users/rosedennis/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [21]:
X_train_df = pd.DataFrame(X_train.toarray(),
                          columns=cvec.get_feature_names())
X_test = cvec.transform(X_test)
X_test_df = pd.DataFrame(X_test.toarray(),
                         columns=cvec.get_feature_names())


In [22]:
l1 = LogisticRegression()
l1.fit(X_train_df, y_train)

/Users/rosedennis/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
print(l1.score(X_train_df, y_train))
l1.score(X_test_df, y_test)

0.9746268656716418


0.6666666666666666